In [1]:
# !pip install gurobipy

In [1]:
from gurobipy import *

## 問題例7 複数のシナリオを生成し同時に解く

In [71]:
def order(a,b,c,d):
  return [a,b,c,d]

In [85]:
# nと初期配置を変更しなければいけない
O1=order(1,2,3,4)
O2=order(2,1,3,4)
O3=order(1,3,2,4)
O4=order(1,2,4,3)
O5=order(2,3,1,4)

O=[O1,O2,O3]

S=[i for i in range(1,5)]
H=[i for i in range(1,5)]
P=[i for i in range(1,5)]
T=[i for i in range(1,7)]
n_=[1,2,4,3]
n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]
n2=[n_[O2[0]-1],n_[O2[1]-1],n_[O2[2]-1],n_[O2[3]-1]]
n3=[n_[O3[0]-1],n_[O3[1]-1],n_[O3[2]-1],n_[O3[3]-1]]
# n3=[n_[O3[0]-1],n_[O3[1]-1],n_[O3[2]-1],n_[O3[3]-1]]
# n4=[n_[O4[0]-1],n_[O4[1]-1],n_[O4[2]-1],n_[O4[3]-1]]
# n5=[n_[O5[0]-1],n_[O5[1]-1],n_[O5[2]-1],n_[O5[3]-1]]

S,H,P,T,n1,n2,O1,O2

([1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5, 6],
 [1, 2, 4, 3],
 [2, 1, 4, 3],
 [1, 2, 3, 4],
 [2, 1, 3, 4])

In [86]:
m=Model("IP3")

In [87]:
x,y,pre,penalty={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(1,len(H)):
    for p in P:
      pre[s,h,p]=m.addVar(vtype="C")
      penalty[s,h,p]=m.addVar(vtype="c")

In [88]:
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])
      # m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n2[p-1])
      # m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n3[p-1])
      # m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n4[p-1])
      # m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n5[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

In [89]:
for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])

In [90]:
for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

# for s in S:
#   for h in range(1,len(H)):
#     for p in P:
#       m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

In [91]:
for s in S:
  for h in range(1,len(H)):

    for o in O:
      for i,p in enumerate(o):
        m.addConstr(penalty[s,h,p]>=quicksum(x[s,h+1,k,len(T)] for k in o[i:])-quicksum(x[s,h,k,len(T)] for k in o[i:]))


    # for p in O1:
    #   m.addConstr(penalty[s,h,p]>=quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))-quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))
    #   m.addConstr(penalty[s,h,p]>=0)

    # for i,p in enumerate(O2):
    #   m.addConstr(penalty[s,h,p]>=quicksum(x[s,h+1,k,len(T)] for k in O2[i:])-quicksum(x[s,h,k,len(T)] for k in O2[i:]))
    #   m.addConstr(penalty[s,h,p]>=0)
      # m.addConstr(penalty[s,h,p]>=pre[s,h,p])
      # print(penalty[s,h,p])


In [92]:
# for i,p in enumerate(O2):
#   print(O2[O2[p]:])

In [93]:
# for i,p in enumerate(O2):
#   print(O2[i:])

In [94]:
m.addConstr(x[1,1,O1[3],1]==1)
m.addConstr(x[1,2,O1[3],1]==1)
m.addConstr(x[1,3,O1[2],1]==1)
m.addConstr(x[2,1,O1[3],1]==1)
m.addConstr(x[2,2,O1[2],1]==1)
m.addConstr(x[2,3,O1[1],1]==1)
m.addConstr(x[3,1,O1[2],1]==1)
m.addConstr(x[3,2,O1[2],1]==1)
m.addConstr(x[4,1,O1[1],1]==1)
m.addConstr(x[4,2,O1[0],1]==1)

# m.addConstr(x[1,1,O[0],1]==1)
# m.addConstr(x[1,2,O[1],1]==1)
# m.addConstr(x[2,1,O[1],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [95]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T)))
+5*quicksum(penalty[s,h,p] for s in S for h in H if h!=len(H) for p in P))

In [96]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1163 rows, 540 columns and 6710 nonzeros
Model fingerprint: 0xc2f8a7e0
Variable types: 96 continuous, 444 integer (444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 447 rows and 193 columns
Presolve time: 0.05s
Presolved: 716 rows, 347 columns, 4007 nonzeros
Variable types: 0 continuous, 347 integer (311 binary)
Found heuristic solution: objective 7.0000000

Root relaxation: objective 1.111111e+00, 373 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.50000    0   87    7.00000    1.50000  78.6%     -    0s
H    0     0                    

In [97]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    # if t==len(T):
    #   if x[s,h,p,t].X>EPS:
    #     print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,p) in penalty:
    if penalty[s,h,p].X>EPS:
      print("penalty[%2s,%2s,%2s]=%3s" %(s,h,p,penalty[s,h,p].X))
  print("the objective function", m.objVal)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 3, 1]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 3, 1]=1.0
x[ 2, 3, 2, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 3, 1]=1.0
x[ 3, 3, 1, 2]=1.0
x[ 4, 1, 2, 1]=1.0
x[ 4, 2, 1, 1]=1.0
x[ 4, 2, 2, 3]=1.0
y[ 2, 4, 2]=1.0
y[ 4, 3, 1]=1.0
the objective function 2.0


In [44]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:

    if t==len(T):
        if x[s,h,p,t].X>EPS:
            print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))


x[ 1, 1, 4, 4]=1.0
x[ 1, 2, 4, 4]=1.0
x[ 1, 3, 3, 4]=1.0
x[ 2, 1, 4, 4]=1.0
x[ 2, 2, 3, 4]=1.0
x[ 2, 3, 2, 4]=1.0
x[ 2, 4, 2, 4]=1.0
x[ 3, 1, 3, 4]=1.0
x[ 3, 2, 3, 4]=1.0
x[ 3, 3, 1, 4]=1.0
